# Demostration:

Here we will demostrate our model, study the predictions and talk about how to further improve our model.


In [14]:
import os
import requests
import datetime
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
import ipywidgets as widgets
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [15]:
#Panda settings
#Pandas will not display all columns in our data when using the head() function without this
pd.set_option('max_columns',50) 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Building Glossary 

This is a very important part. When making individual predictions, we need to encode and scale our data before running the chosen model. If we encode data from an new instance the model will predict without problems but the encoding will not match the encoding that was done to the dataframe and that was fed to the model. 

In order to deal with this we need to map avery single instance with its correspondent encoded value by building a glossary:

In [16]:
with open('x_y_z.pickle', 'rb') as data:
    Data = pickle.load(data)
    
X = Data[0]
y = Data[1]
z = Data[2]
X_raw = Data[3]
X = X[["hour","street", "month", "day", "LATITUDE", "LONGITUDE", "Temperature"]]

X_raw = X_raw[["hour","street", "month", "day", "LATITUDE", "LONGITUDE", "Temperature"]]

Identifiying minimum and maximum values from the numerical features

In [17]:
print(X_raw['LATITUDE'].min())
print(X_raw['LATITUDE'].max())

print(X_raw['LONGITUDE'].min())
print(X_raw['LONGITUDE'].max())

print(X_raw['day'].min())
print(X_raw['day'].max())

print(X_raw['Temperature'].min())
print(X_raw['Temperature'].max())

print(X_raw['hour'].min())
print(X_raw['hour'].max())


38.82
38.995
-77.114
-76.91
1.0
31.0
9.0
98.0
0.0
23.0


In [18]:
X_raw.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 277795 entries, 2010-06-11 04:00:00 to 2018-06-11 03:00:00
Data columns (total 7 columns):
hour           277795 non-null float64
street         277795 non-null object
month          277795 non-null float64
day            277795 non-null float64
LATITUDE       277795 non-null float64
LONGITUDE      277795 non-null float64
Temperature    277795 non-null float64
dtypes: float64(6), object(1)
memory usage: 17.0+ MB


Creating the encoded - unencoded translator function:

In [19]:
def decoder(encoded, unencoded):
    Glossary = {}
    for i, j in zip(unencoded, encoded):
        Glossary[i] = j
        
    return (Glossary)

In [42]:
gloss = decoder(X['LATITUDE'], X_raw['LATITUDE'])

In [45]:
print(gloss)

{38.897: 77, 38.942: 122, 38.886: 66, 38.889: 69, 38.906: 86, 38.918: 98, 38.898: 78, 38.926: 106, 38.95: 130, 38.915: 95, 38.894: 74, 38.863: 43, 38.875: 55, 38.957: 137, 38.871: 51, 38.885: 65, 38.844: 24, 38.904: 84, 38.908: 88, 38.89: 70, 38.967: 147, 38.907: 87, 38.954: 134, 38.9: 80, 38.88: 60, 38.905: 85, 38.925: 105, 38.928: 108, 38.931: 111, 38.948: 128, 38.934: 114, 38.903: 83, 38.956: 136, 38.833: 13, 38.909: 89, 38.944: 124, 38.919: 99, 38.91: 90, 38.883: 63, 38.929: 109, 38.881: 61, 38.895: 75, 38.945: 125, 38.941: 121, 38.899: 79, 38.892: 72, 38.872: 52, 38.901: 81, 38.936: 116, 38.847: 27, 38.94: 120, 38.927: 107, 38.92: 100, 38.829: 9, 38.854: 34, 38.916: 96, 38.825: 5, 38.923: 103, 38.842: 22, 38.914: 94, 38.868: 48, 38.873: 53, 38.939: 119, 38.911: 91, 38.937: 117, 38.958: 138, 38.888: 68, 38.922: 102, 38.822: 2, 38.946: 126, 38.952: 132, 38.893: 73, 38.884: 64, 38.869: 49, 38.848: 28, 38.857: 37, 38.93: 110, 38.983: 163, 38.935: 115, 38.949: 129, 38.832: 12, 38.932: 

## Dine Out, Order_in:

### Importing the estimators:

In [21]:
with open('offensegroup_upsampled_model.pickle', 'rb') as model:
    
    estimators = pickle.load(model)

In [22]:
# print(estimators) # 0: knn, 1: RandomForest, 2: SGD, 3: BaggingClassifier, 4: PassiveAgressiveClassifier

### Creating the user interface: 

In [38]:
hour = widgets.BoundedFloatText(value = 5, min=0, max=23,  description='Hour:')
street = widgets.Text(value = 'ridge road se',  description='Street Name:')
month = widgets.BoundedFloatText(value = 6, min = 1, max = 12, description='Month:')
day = widgets.BoundedFloatText(value = 5, min = 1, max = 31, description='Day:')
lat = widgets.FloatText(value = 38.886,  min = 38.820,  max = 38.995 , step=0.001,  description='LATITUDE:')
lon = widgets.FloatText(value = -76.949, min = -77.114, max = -76.910, step=0.001,  description='LONGITUDE:')
temp = widgets.BoundedFloatText(value = 62, min = -10, max = 130, description='Temperature:')





In [39]:
def funct1(hour, street, month, day, lat, lon, temp):
    
    dictionary = {'hour': [],'street': [],
         'month': [], 'day': [], 'LATITUDE': [], 
        'LONGITUDE': [], 'Temperature': []}
    
    Hour = decoder(X['hour'], X_raw['hour'])
    Street = decoder(X['street'], X_raw['street'])
    Month = decoder(X['month'], X_raw['month'])
    Day = decoder(X['day'], X_raw['day'])
    Lat = decoder(X['LATITUDE'], X_raw['LATITUDE'])
    Lon = decoder(X['LONGITUDE'], X_raw['LONGITUDE'])
    Temp = decoder(X['Temperature'], X_raw['Temperature'])
    
    dictionary['hour'] = [Hour[hour.value]]
    dictionary['street'] = [Street[street.value]]
    dictionary['month'] = [Month[month.value]]
    dictionary['day'] = [Day[day.value]]
    dictionary['LATITUDE'] = [Lat[lat.value]]
    dictionary['LONGITUDE'] = [Lon[lon.value]]
    dictionary['Temperature'] = [Temp[temp.value]]
    
    return dictionary
    
    #return Hour[hour.value], Street[street.value], Month[month.value], Day[day.value], Lat[lat.value], Lon[lon.value], Temp[temp.value]
    


In [40]:
def funct2(b):
    try:
        scaler = StandardScaler()
        data = funct1(hour, street, month, day, lat, lon, temp)
        data = pd.DataFrame(data)
        #data = np.array(data)
        #data = data.reshape(-1,1)
        data = scaler.fit(data).transform(data)
        results = estimators[1].predict(data)
    
        final_results = []
    
        for i in results:
        
            if i == 0:
                i = 'Dine out'
            else:
                i = 'Order In'
        
            final_results.append(i)
        print (final_results)
    
    except:
        
        print("Value is not in the database, we cannot predict crime safety for this particular place and time, Please try again")
    

In [41]:
button = widgets.Button(description="Predict!")

display(hour,street,month,day,lat,lon,temp,button)
button.on_click(funct2)


BoundedFloatText(value=5.0, description='Hour:', max=23.0)

Text(value='ridge road se', description='Street Name:')

BoundedFloatText(value=6.0, description='Month:', max=12.0, min=1.0)

BoundedFloatText(value=5.0, description='Day:', max=31.0, min=1.0)

FloatText(value=38.886, description='LATITUDE:', step=0.001)

FloatText(value=-76.949, description='LONGITUDE:', step=0.001)

BoundedFloatText(value=62.0, description='Temperature:', max=130.0, min=-10.0)

Button(description='Predict!', style=ButtonStyle())

['Dine out']
['Dine out']
['Dine out']
['Dine out']
['Dine out']
['Dine out']


In [ ]:
38.8980464,-77.0354491

### Testing on the whole dataset:

In [ ]:
38.8658045,-76.9913079

In [27]:
scaler = StandardScaler()
test = scaler.fit(X).transform(X)

t = estimators[3].predict(X)
t = pd.Series(t)

In [28]:
t.value_counts()

0    218347
1     59448
dtype: int64

It is very interesting that here the only model that performed consistent predictions was the bagging classifier.
Ihe "Order In"/"Dine Out" proportion is very close with the real "offensegroup" variable

KNN was the most accurate model but it only predict 0('Dine Out') values.

# Dine out/metro, Dine out, Order In

In [16]:
with open('ucrrank_upsampled_model.pickle', 'rb') as model:
    
    estimators_ucr = pickle.load(model)


In [17]:
def funct3(b):
    #try:
    scaler = StandardScaler()
    data = funct1(hour, street, month, day, lat, lon, temp)
    data = pd.DataFrame(data)
    data = scaler.fit(data).transform(data)
    results_ucr = estimators_ucr[3].predict(data)
    
    final_results_ucr = []
    
    for i in results_ucr:
        
        if i == 3:
            i = 'Order In'
        elif i == 2:
            i = 'Dine Out/Metro'
        else:
            i = 'Dine Out'
        
        
        final_results_ucr.append(i)
    print(final_results_ucr)
    
    #except:
        
     #   print("Value is not in the database, we cannot predict crime safety for this particular place and time, Please try again")
    

In [18]:
hour = widgets.BoundedFloatText(value = 5, min=0, max=23,  description='our:')
street = widgets.Text(value = 'ridge road se',  description='Street Name:')
month = widgets.BoundedFloatText(value = 6, min = 1, max = 12, description='Month:')
day = widgets.BoundedFloatText(value = 5, min = 1, max = 31, description='Day:')
lat = widgets.FloatText(value = 38.886,  min = 38.820,  max = 38.995 , step=0.001,  description='LATITUDE:')
lon = widgets.FloatText(value = -76.949, min = -77.114, max = -76.910, step=0.001,  description='LONGITUDE:')
temp = widgets.BoundedFloatText(value = 62, min = -10, max = 130, description='Temperature:')


In [19]:
button = widgets.Button(description="Predict!")

display(hour,street,month,day,lat,lon,temp,button)
button.on_click(funct3)


BoundedFloatText(value=5.0, description='our:', max=23.0)

Text(value='ridge road se', description='Street Name:')

BoundedFloatText(value=6.0, description='Month:', max=12.0, min=1.0)

BoundedFloatText(value=5.0, description='Day:', max=31.0, min=1.0)

FloatText(value=38.886, description='LATITUDE:', step=0.001)

FloatText(value=-76.949, description='LONGITUDE:', step=0.001)

BoundedFloatText(value=62.0, description='Temperature:', max=130.0, min=-10.0)

Button(description='Predict!', style=ButtonStyle())

['Dine Out']


In [20]:
scaler = StandardScaler()
test = scaler.fit(X).transform(X)

r = estimators_ucr[3].predict(X)
r = pd.Series(r)

In [21]:

#r.head(1000)
r.value_counts()

2    277518
1       277
dtype: int64

# Classification reports: (from the last Kfold)

In [22]:
reports = [ 'offensegroup_upsampled_report.pickle', 'offensegroup_downsampled_report.pickle',
           'ucrrank_upsampled_report.pickle', 'ucrrank_downsampled_report.pickle']
classification_matrix = []



with open('offensegroup_upsampled_report.pickle', 'rb') as upsampled_model_offensegroup:
    
        estimator_upsampled_offensegroup = pickle.load(upsampled_model_offensegroup)
        
with open('ucrrank_upsampled_report.pickle', 'rb') as upsampled_model_ucrrank:
    
        estimator_upsampled_ucrrank = pickle.load(upsampled_model_ucrrank)

list order: [predicted_knn, predicted_rnf, predicted_sgd, predicted_baggin, predicted_pas, report_knn, report_rnf, report_sgd, report_baggin, report_pas] 

In [23]:
print("Classification Report for Upsampled Knn on offensegroup: \n\n"+estimator_upsampled_offensegroup[5])
print("==============================================================================")
print("Classification Report for Upsampled RandomForest on offensegroup: \n\n"+estimator_upsampled_offensegroup[6])
print("==============================================================================")
# 0 Nonviolent, 1 is violent.
print("Classification Report for Upsampled BaggingClassifier on offensegroup: \n\n"+estimator_upsampled_offensegroup[8])

Classification Report for Upsampled Knn on offensegroup: 

             precision    recall  f1-score   support

          0       0.68      0.63      0.65     19218
          1       0.66      0.70      0.68     19218

avg / total       0.67      0.67      0.67     38436

Classification Report for Upsampled RandomForest on offensegroup: 

             precision    recall  f1-score   support

          0       0.64      0.94      0.76     19218
          1       0.89      0.48      0.62     19218

avg / total       0.77      0.71      0.69     38436

Classification Report for Upsampled BaggingClassifier on offensegroup: 

             precision    recall  f1-score   support

          0       0.63      0.64      0.63     19218
          1       0.63      0.62      0.63     19218

avg / total       0.63      0.63      0.63     38436



# 50 Restaurant locations:

In [295]:
locations = pd.read_csv('locations.csv')

### Add current year, month, day and hour to the locations dataframe

In [296]:
now = datetime.datetime.now()

print("Current date and time using str method of datetime object:")
print(str(now))

Current date and time using str method of datetime object:
2018-06-28 03:52:26.684411


In [297]:
def label_locations_month (row):
    month = int(now.month)
    return month

locations['month'] = locations.apply (lambda row: label_locations_month (row),axis=1)

In [298]:
def label_locations_day (row):

    day = int(now.day)
    
    return day
locations['day'] = locations.apply (lambda row: label_locations_day (row),axis=1)

In [299]:
def label_locations_hour (row):

    hour = int(now.hour)
    
    return hour

In [300]:
locations['hour'] = locations.apply (lambda row: label_locations_hour (row),axis=1)

### Add current temperature to locations dataframe

In [301]:
URL = "http://api.openweathermap.org/data/2.5/weather?q=Washington,us&units=imperial&appid=c267487c712e3fa110ff1d1b9eccc88b"

def fetch_data(fname="weather"):
    """
    Helper method to retrieve the ML Repository dataset.
    """
    response = requests.get(URL)
    outpath  = os.path.abspath(fname)
    with open(outpath, 'wb') as f:
        f.write(response.content)
    
    return outpath

DATA = fetch_data()


In [302]:
import json

with open('weather') as json_data:
    current_weather = json.load(json_data)
    print(current_weather)

{'coord': {'lon': -77.04, 'lat': 38.9}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 75.2, 'pressure': 1010, 'humidity': 83, 'temp_min': 71.6, 'temp_max': 78.8}, 'visibility': 16093, 'wind': {'speed': 11.41, 'deg': 220}, 'clouds': {'all': 90}, 'dt': 1530170160, 'sys': {'type': 1, 'id': 3131, 'message': 0.005, 'country': 'US', 'sunrise': 1530179121, 'sunset': 1530232656}, 'id': 4366164, 'name': 'Washington DC.', 'cod': 200}


In [303]:
def label_locations_temp (row):

    temperature = current_weather['main']['temp']
    
    return temperature

In [304]:
locations['temperature'] = locations.apply (lambda row: label_locations_temp (row),axis=1)

In [305]:
temperature = locations['temperature'][0]
temperature

75.2

In [306]:
locations['temperature'] = locations['temperature'].round(decimals = 0)

In [307]:
locations.head(5)

,id,address,business_name,categories,category01,city,coordinates,country,image_url,latitude,location_id,longitude,phone,rating,review_count,state,street_name,street_number,url,zip_code,month,day,hour,temperature
0,739,1337 14th St NW,ChurchKey,* American (New) * Lounges,American (New),"Washington, DC","38.9084128488013,-77.0318697756855",US,https://s3-media1.fl.yelpcdn.com/bphoto/yBfvlN...,38.908413,si_pqK5Ogi5MZdO-aN_gXw,-77.031870,(202) 567-2576,4.0,1310.0,DC,14th st nw,1337.0,https://www.yelp.com/biz/churchkey-washington?...,20005.0,6,28,3,75.0
1,719,1924 Pennsylvania Ave NW,Founding Farmers - DC,* American (Traditional) * Coffee & Tea * Bre...,American (Traditional),"Washington, DC","38.900313,-77.044533",US,https://s3-media2.fl.yelpcdn.com/bphoto/AWpidG...,38.900313,VA8aPObRynlwR1TGzbzraQ,-77.044533,(202) 822-8783,4.0,10596.0,DC,pennsylvania ave nw,1924.0,https://www.yelp.com/biz/founding-farmers-dc-w...,20006.0,6,28,3,75.0
2,720,675 15th St NW,Old Ebbitt Grill,* Bars * American (Traditional) * Breakfast &...,Bars,"Washington, DC","38.898005,-77.033362",US,https://s3-media2.fl.yelpcdn.com/bphoto/KBCezp...,38.898005,iyBbcXtQSBfiwFQZwVBNaQ,-77.033362,(202) 347-4800,4.0,6394.0,DC,15th st nw,675.0,https://www.yelp.com/biz/old-ebbitt-grill-wash...,20005.0,6,28,3,75.0
3,721,1320 14th St NW,The Pig,* American (Traditional) * Cocktail Bars,American (Traditional),"Washington, DC","38.90793,-77.03229",US,https://s3-media3.fl.yelpcdn.com/bphoto/tfq_cx...,38.907930,B-NQjTjaIPURVtpUQe0-3w,-77.032290,(202) 290-2821,4.0,1387.0,DC,14th st nw,1320.0,https://www.yelp.com/biz/the-pig-washington-2?...,20005.0,6,28,3,75.0
4,722,2010 P St NW,Tiki Taco,* Tacos * Hawaiian * Shaved Ice,Tacos,"Washington, DC","38.90939,-77.04541",US,https://s3-media2.fl.yelpcdn.com/bphoto/zyWPCJ...,38.909390,HVDw0WV-tLIhF8mGYNq1vg,-77.045410,(202) 986-2121,4.5,31.0,DC,p st nw,2010.0,https://www.yelp.com/biz/tiki-taco-washington?...,20036.0,6,28,3,75.0


### Prepare dataframes for demo

In [308]:
locations_business = locations.drop(['id', 'categories', 'category01', 'city', 'coordinates', 'country', 'image_url', 'location_id', 'phone', 'rating', 'review_count', 'state', 'street_name', 'street_number', 'url', 'zip_code'], axis=1)
locations_ML = locations.drop(['business_name'], axis=1)
locations_ML = locations_ML[['longitude', 'latitude', 'month', 'day', 'hour', 'temperature', 'street_name']]
locations_ML = locations_ML.rename(columns={"longitude": "LONGITUDE", "latitude": "LATITUDE", "temperature": "Temperature",
                                           "hour": "hour", "day": "day", "street_name": "street"})

In [309]:
locations_ML = locations_ML.dropna()
locations_ML['LATITUDE'] = locations_ML['LATITUDE'].round(decimals = 3)
locations_ML['LONGITUDE'] =locations_ML['LONGITUDE'].round(decimals = 3)
locations_ML['street']= locations_ML['street'].replace("st", "St")

In [310]:
lis = []
for row in locations_ML['street']:
    row = row.replace("st", "St")
    lis.append(row)
locations_ML['street'] = lis
locations_ML['street'] = locations_ML['street'].replace(["pennsylvania ave nw", "wisconsin ave nw","sherman ave nw",
                                                         "connecticut ave nw", "new hampshire ave nw","massachusetts ave nw",
                                                        "georgia ave nw", "rhode island ave nw", "pennsylvania ave se"],
                                                        ["pennsylvania avenue nw", "wisconsin avenue nw","sherman avenue nw",
                                                        "connecticut avenue nw", "new hampshire avenue nw", "massachusetts avenue nw",
                                                        "georgia avenue nw", "rhode island avenue nw", "pennsylvania avenue se"])

In [311]:
locations_ML = locations_ML.drop(locations_ML.index[16])
locations_ML = locations_ML.drop(locations_ML.index[31])
#locations_ML = locations_ML.drop(locations_ML.index[31])

In [312]:
locations_ML

,LONGITUDE,LATITUDE,month,day,hour,Temperature,street
0,-77.032,38.908,6,28,3,75.0,14th St nw
1,-77.045,38.900,6,28,3,75.0,pennsylvania avenue nw
2,-77.033,38.898,6,28,3,75.0,15th St nw
3,-77.032,38.908,6,28,3,75.0,14th St nw
4,-77.045,38.909,6,28,3,75.0,p St nw
5,-77.064,38.907,6,28,3,75.0,wisconsin avenue nw
6,-77.026,38.926,6,28,3,75.0,sherman avenue nw
7,-77.060,38.901,6,28,3,75.0,k St nw
8,-77.051,38.906,6,28,3,75.0,24th St nw
9,-77.032,38.911,6,28,3,75.0,14th St nw


In [313]:
def proper_encode(hour, street, month, day, lat, lon, temp):
    
    dictionary = {'hour': [],'street': [],
         'month': [], 'day': [], 'LATITUDE': [], 
        'LONGITUDE': [], 'Temperature': []}
    
    Hour = decoder(X['hour'], X_raw['hour'])
    Street = decoder(X['street'], X_raw['street'])
    Month = decoder(X['month'], X_raw['month'])
    Day = decoder(X['day'], X_raw['day'])
    Lat = decoder(X['LATITUDE'], X_raw['LATITUDE'])
    Lon = decoder(X['LONGITUDE'], X_raw['LONGITUDE'])
    Temp = decoder(X['Temperature'], X_raw['Temperature'])
    
    dictionary['hour'] = [Hour[hour.value]]
    dictionary['street'] = [Street[street.value]]
    dictionary['month'] = [Month[month.value]]
    dictionary['day'] = [Day[day.value]]
    dictionary['LATITUDE'] = [Lat[lat.value]]
    dictionary['LONGITUDE'] = [Lon[lon.value]]
    dictionary['Temperature'] = [Temp[temp.value]]
    
    return dictionary

In [314]:
def proper_encode(df):
    dictionary = {'hour': [],'street': [],
         'month': [], 'day': [], 'LATITUDE': [], 
        'LONGITUDE': [], 'Temperature': []}
    
    Hour = decoder(X['hour'], X_raw['hour'])
    Street = decoder(X['street'], X_raw['street'])
    Month = decoder(X['month'], X_raw['month'])
    Day = decoder(X['day'], X_raw['day'])
    Lat = decoder(X['LATITUDE'], X_raw['LATITUDE'])
    Lon = decoder(X['LONGITUDE'], X_raw['LONGITUDE'])
    Temp = decoder(X['Temperature'], X_raw['Temperature'])
    
    for colname, col in df.iteritems():
        dictionary['hour'] = [Hour[col]]

In [315]:
def proper_encode(hour, street, month, day, lat, lon, temp):
    
    dictionary = {'hour': [],'street': [],
         'month': [], 'day': [], 'LATITUDE': [], 
        'LONGITUDE': [], 'Temperature': []}
    
    Hour = decoder(X['hour'], X_raw['hour'])
    Street = decoder(X['street'], X_raw['street'])
    Month = decoder(X['month'], X_raw['month'])
    Day = decoder(X['day'], X_raw['day'])
    Lat = decoder(X['LATITUDE'], X_raw['LATITUDE'])
    Lon = decoder(X['LONGITUDE'], X_raw['LONGITUDE'])
    Temp = decoder(X['Temperature'], X_raw['Temperature'])
    
    for i,j,k,l,m,n, o in zip(hour, street, month, day, lat, lon, temp):
        
        dictionary['hour'].append(Hour[i])
        dictionary['street'].append(Street[j])
        dictionary['month'].append(Month[k])
        dictionary['day'].append(Day[l])
        dictionary['LATITUDE'].append(Lat[m])
        dictionary['LONGITUDE'].append(Lon[n])
        dictionary['Temperature'].append(Temp[o])
    
    return dictionary

In [316]:
locations_ML = proper_encode(locations_ML['hour'],locations_ML['street'],locations_ML['month'],
                             locations_ML['day'], locations_ML['LATITUDE'], 
                             locations_ML['LONGITUDE'], locations_ML['Temperature'] )

In [317]:
locations_ML

{'hour': [17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17],
 'street': [251,
  2332,
  258,
  251,
  2297,
  3067,
  2779,
  1290,
  304,
  251,
  2949,
  488,
  912,
  251,
  232,
  471,
  1236,
  1402,
  1155,
  1236,
  488,
  267,
  2425,
  477,
  327,
  1155,
  1402,
  251,
  464,
  484,
  1489,
  2332,
  2333,
  1433,
  912,
  267,
  251,
  1290,
  994,
  251,
  1130,
  912,
  2970,
  1402,
  484,
  1811,
  258,
  1041],
 'month': [8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8],
 'day': [20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20,
  20

In [318]:
locations_ML = pd.DataFrame(locations_ML)

In [319]:
locations_ML

,LATITUDE,LONGITUDE,Temperature,day,hour,month,street
0,88,122,65,20,17,8,251
1,80,135,65,20,17,8,2332
2,78,123,65,20,17,8,258
3,88,122,65,20,17,8,251
4,89,135,65,20,17,8,2297
5,87,154,65,20,17,8,3067
6,106,116,65,20,17,8,2779
7,81,150,65,20,17,8,1290
8,86,141,65,20,17,8,304
9,91,122,65,20,17,8,251


In [334]:
scaler = StandardScaler()
test = scaler.fit(locations_ML).transform(locations_ML)

t = estimators[3].predict(locations_ML)
t = pd.Series(t)

In [335]:
t

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
dtype: int64